# Lecture 5: FEM, adaptivity

## Linear Elasticity

* Consider a domain $\Omega$ as shown here:
<img src="fig/2Delastic_domain_ext.png">
We assume it is a piece of elastic material (similar to a beam) of this shape. Fixed at the left end and the force $P$ is applied to its right end.

* Boundary:
    - leftmost: $\Gamma_1$
    - rightmost: $\Gamma_3$
    - rest: $\Gamma_2$
* Displacement: $(u,v)$ (in the $x$ and $y$ coord)

* Elasticity tensors:
$$
\begin{align*}
C_{11} &= \begin{pmatrix}
\frac{E (1-\nu)}{(1-2\nu)(1+\nu)} & 0 \\ 0 & \frac{E}{2(1+\nu)} \end{pmatrix},
&\qquad
C_{12} &= \begin{pmatrix} 0 & \frac{E \nu}{(1-2\nu)(1+\nu)}  \\ \frac{E}{2(1+\nu)} & 0 \end{pmatrix},
\\
C_{21} &= C_{12}^{\rm T},
&\qquad
C_{22} &= \begin{pmatrix} \frac{E}{2(1+\nu)} & 0 \\ 0 & \frac{E (1-\nu)}{(1-2\nu)(1+\nu)} \end{pmatrix}
\end{align*}
$$
Here $E$ is called the modulus of elasticity and $\nu$ is called the Poisson ratio.
(In fact, $C$ is a four-dimensional tensor, denoted by $C$, most likely, after Cauchy.)

* Equations:
$$
\begin{align*}
-{\rm div} (C_{11} \nabla u + C_{12} \nabla v) &= 0
\qquad\text{in }\Omega
\\
-{\rm div} (C_{21} \nabla u + C_{22} \nabla v) &= 0
\qquad\text{in }\Omega
\\
u = v &= 0 \qquad\text{on }\Gamma_1
\\
C_{11} u_n + C_{12} v_n = C_{21} u_n + C_{22} v_n &= 0 \qquad\text{on }\Gamma_2
\\
C_{11} u_n + C_{12} v_n &= 0 \qquad\text{on }\Gamma_3
\\
C_{21} u_n + C_{22} v_n &= \frac{P}{H W} \qquad\text{on }\Gamma_3,
\end{align*}
$$
where $u_n$ and $v_n$ are normal derivatives of the solution.

* Variational formulation:
    - test functions $p=p(x,y)$ and $q = q(x,y)$
    - Dirichet BCs: $p=q=0$ on $\Gamma_1$
    - Variational form:
$$
\int_{\Omega} \big[
    (C_{11} \nabla u + C_{12} \nabla v)\cdot \nabla p
    +
    (C_{21} \nabla u + C_{22} \nabla v)\cdot \nabla q
\big]
= \frac{P}{H W}  \int_{\Gamma_3} q
\qquad\forall p, q
$$

* A good way to go about this problem is to think that your space consists of vector-valued functions $(u,v)\in\mathbb R^2$. Thus, at each node you have two basis functions: such that $(u,v) = (1,0)$ for that node, and $(u,v) = (0,1)$ for that node.

## Adaptivity

Consider a 1D equation:
$$ \int_0^1 u' v' = \int_0^1 f v $$
and substitute there the approximate solution (here ${\rm Res}(v)$ is the residual):
$$
\begin{align*}
{\rm Res}(v) :=& \int _0^1 f v - \int _0^1 u_h' v'
\\=&~~ \sum_T \int_T \big(f v - u_h' v'\big)
\\=&~~ \sum_T \int_T \big(f v + u_h'' v\big) + \sum _i \big(u _h'(x _i+0) - u _h'(x _{i+1}-0)\big) v(x_i)
\end{align*}
$$
In any dimension one has:
$$
{\rm Res}(v) =  \sum_T \int_T \big(f v + \Delta u_h v\big) + \sum _{\gamma \in \Gamma}\int _\gamma [\nabla u _h] _{\gamma} v,
$$
where $\gamma$ is the edge that $T _1$ and $T _2$ share, and
$$[\nabla u _h] _{\gamma} = (\nabla u _h) _{T _1} \cdot n _1 + (\nabla u _h) _{T _2} \cdot n _2$$
<img src="fig/FEM_adapt_gamma.png">

With a few more manipulations, we can derive
$$
\begin{align*}
\|\nabla (u_h-u)\|^2
&\leq
С \sum_T h_T^2 \|f + \Delta u_h\| _{L^2}^2 + С \sum _{\gamma} h _{\gamma} \big\|[\nabla u _h] _{\gamma} \big\|_{\gamma}^2
\\ &=~~
С \sum_T h_T^2 \|f\| _{L^2}^2 + С \sum _{\gamma} h _{\gamma} \big\|[\nabla u _h] _{\gamma} \big\|_{L^2(\gamma)}^2
\end{align*}
$$
(used that $\Delta u_h = 0$ in any $T$).

Hence the strategy:

* Estimate error contributions for each non-Dirichlet edge $\gamma$
* Mark some of the edges for refinement (e.g., those with top 20% of error estimator)
* Refine
* Repeat

### Demo:

We solve
$$
\begin{align*}
\Delta u &= 1 ~~~~ \text{in }\Omega \\
u |_ \Gamma &= 0 ~~~~ \text{in }\Gamma
\end{align*}
$$
In this region:
<img src='fig/Lshape_refin.png'>

### References

* [Thomas Gratsch, Klaus-Jurgen Bathe, A posteriori error estimation techniques in practical finite element analysis](http://web.mit.edu/kjb/www/Principal_Publications/A_posteriori_Error_Estimation_Techniques_in_Practical_Finite_Element_Analysis.pdf)


### Goal-oriented adaptivity

* Suppose we are interested only in
$$
Q(u) = \int _{\Omega} u
$$
(quantity of interest). Our error is:
$$
{\rm err} = Q(u _h)-Q(u)
$$

* Consider $z$ such that
$$
Q(u) = A(u, z) ~~~~ \forall u
$$
Then ${\rm err} = A(u _h,z)-A(u,z) = A(u _h,z)-F(z)$

* Unfortunately, it is useless to approximate $z$ with $z_h$:
$$
{\rm err} \approx A(u _h,z _h)-F(z _h) = 0
$$

* Hence find $\tilde{z}$ which approximates $z$ better than $z_h$ (variants: with higher-order approximation, or with denser mesh):
$$
{\rm err} \approx A(u _h,\tilde{z}) - F(\tilde{z}) = A(u _h,\tilde{z}-\tilde{z} _h) - F(\tilde{z}-\tilde{z} _h) = \int _{\Omega} \big(\nabla u _h\cdot \nabla(\tilde{z}-\tilde{z} _h) - f (\tilde{z}-\tilde{z} _h) \big),
$$
where $\tilde{z}_h$ is the approximation of $\tilde{z}$.
Then one refines where $\nabla u _h\cdot \nabla(\tilde{z}-\tilde{z} _h) - f (\tilde{z}-\tilde{z} _h)$ is large.